In [5]:
import pickle
import seaborn as sns
sns.set()
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import glob
from scipy.signal import savgol_filter
import imageio
import pickle
import glob
import cv2
from moviepy.editor import *

In [29]:
index = list(range(0,1050,50)) + list(range(1200,2001,200))
print index

[0, 50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1200, 1400, 1600, 1800, 2000]


In [86]:
index = [60, 80, 100, 120, 140, 150, 160, 180, 250, 300, 350, 400, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1200, 1400, 1600, 1800, 2000]

In [44]:
index = [500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1200, 1400, 1600, 1800, 2000]

In [89]:
index = [60, 80, 100, 120, 140, 150, 160, 180, 250, 300, 350,  400, 500, 600, 700, 800, 900, 1000, 1400, 2000]

In [81]:
index = [0,60, 80, 100, 120, 140, 160, 180]
print index

[0, 60, 80, 100, 120, 140, 160, 180, 200]


In [65]:
class Prep_Data():
    def __init__(self, path=None, filter_size=401):
        with open(path, "rb") as p:
            data = pickle.load(p)
        
        self.vels = data["vels"]
        self.kds = data["kds"]
        self.dvs = data["dvs"]        
        self.states = data["states"]
        if self.vels.shape[0]<=filter_size:
            self.vels_smooth = self.vels
        else:
            self.vels_smooth = savgol_filter(self.vels, filter_size, 3)
        self.label = path.split("data/")[-1].split("_/")[0]

In [66]:
def make_vel_plot(vels, dvels):
    sns.set_style("darkgrid")
    sns.set_context("paper", font_scale=2)
    blues = sns.dark_palette("blue", 8)
    
    f, ax = plt.subplots(figsize=(10,5))
    times = np.arange(vels.shape[0])*0.002
    ax.plot(np.arange(dvels.shape[0])*0.002, dvels, "--", color=blues[3], linewidth=4, label="desired velocity")
    ax.plot(times, vels, "-", color=blues[7], linewidth=1, label="hip velocity")
    if vels.shape[0] > 2:
        ax.plot(times[vels.shape[0]-1], vels[-1], "o", color=blues[7], linewidth=6, label="current hip velocity")
    else:
        ax.plot(0., 0., "o", color=blues[7], linewidth=6, label="current hip velocity")
    ax.set_ylim(0., 1.5)
    ax.set_xlim(0, 21)
    ax.legend(loc=4)
    
    f.text(0.5, 0.05, 'time(s)', ha='center')
    f.text(0.05, 0.5, 'hip velocity (m/s)', va='center', rotation='vertical')
    
    return f

In [82]:
for ind in index:
    base_path = "log/"+str(ind)+"/data/vary_0.9_1.2_0.9_1.4_/"
    filename = glob.glob(base_path+"o*.mp4")[0]
    print filename
    vid = imageio.get_reader(filename, 'ffmpeg')
    print vid
    images = []
    for im in vid:
        images += [im]
    images = np.array(images)
    print images.shape
    vid = None
    
    data = Prep_Data(path=base_path+"vary_0.9_1.2_0.9_1.4_.pkl")
    
    video_path = base_path + "training_frames/"
    if not os.path.exists(video_path):
        os.makedirs(video_path)
    for i in range(images.shape[0]):
        if i % 25 == 0:
            fig = make_vel_plot(data.vels_smooth[0:i], data.dvs)
            plt.savefig(video_path+str("%05d" % i)+".jpg")
            plt.clf()
            img = imageio.imread(video_path+str("%05d" % i)+".jpg")
            img = cv2.resize(img, (500, 250))
            frame = np.concatenate([images[i], img], axis=0)
            frame = cv2.putText(frame, "Training Step "+str(ind),
                                (0,30), cv2.FONT_HERSHEY_SIMPLEX,
                                1, (250,255,255), 2)
            imageio.imwrite(video_path+str("%05d" % i)+".jpg", frame)


log/0/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.0.18936.video000000.mp4
(44, 500, 500, 3)
log/60/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.6.18936.video000000.mp4
(280, 500, 500, 3)
log/80/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.8.18936.video000000.mp4
(360, 500, 500, 3)
log/100/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.10.18936.video000000.mp4
(365, 500, 500, 3)
log/120/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.12.18936.video000000.mp4
(274, 500, 500, 3)
log/140/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.14.18936.video000000.mp4
(425, 500, 500, 3)
log/160/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.16.18936.video000000.mp4
(320, 500, 500, 3)
log/180/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.18.18936.video000000.mp4
(435, 500, 500, 3)
log/200/data/vary_0.9_1.2_0.9_1.4_/openaigym.video.4.24461.video000000.mp4


CannotReadFrameError: Could not read frame 587:
Frame is 0 bytes, but expected 750000.
=== stderr ===
ffmpeg version N-86111-ga441aa90e8-static http://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2017 the FFmpeg developers
  built with gcc 5.4.1 (Debian 5.4.1-8) 20170304
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc-5 --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gray --enable-libass --enable-libfreetype --enable-libfribidi --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-librtmp --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libzimg
  libavutil      55. 63.100 / 55. 63.100
  libavcodec     57. 96.101 / 57. 96.101
  libavformat    57. 7

In [90]:
slow_frames = []
fast_frames = []
for ind in index:
    base_path = "log/"+str(ind)+"/data/vary_0.9_1.2_0.9_1.4_/"
    video_path = base_path + "training_frames/"
    for i in range(10000):
        if i % 25 == 0 and os.path.isfile(video_path+str("%05d" % i)+".jpg"):
            frame = imageio.imread(video_path+str("%05d" % i)+".jpg")
            if ind < 300:
                slow_frames += [frame]
            else:
                fast_frames += [frame]

In [91]:
slow_clip = ImageSequenceClip(slow_frames, fps=20)
fast_clip = ImageSequenceClip(fast_frames, fps=80)
clip = concatenate_videoclips([slow_clip, fast_clip])
clip.write_videofile("log/training_demo.mp4", audio=False)

[MoviePy] >>>> Building video log/training_demo.mp4
[MoviePy] Writing video log/training_demo.mp4


100%|██████████| 5762/5762 [00:15<00:00, 377.25it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: log/training_demo.mp4 



In [100]:
base_path = "log/2000/data/vary_1.4_0.8_/"

In [101]:
filename = glob.glob(base_path+"*.mp4")[0]
vid = imageio.get_reader(filename, 'ffmpeg')
images = []
for i, im in enumerate(vid):
    images += [im]
images = np.array(images)

In [102]:
print images.shape

(10000, 500, 500, 3)


In [103]:
class Prep_Data():
    def __init__(self, path=None, filter_size=401):
        with open(path, "rb") as p:
            data = pickle.load(p)
        
        self.vels = data["vels"]
        self.kds = data["kds"]
        self.dvs = data["dvs"]        
        self.states = data["states"]
        self.vels_smooth = savgol_filter(self.vels, filter_size, 3)
        self.label = path.split("data/")[-1].split("_/")[0]

In [104]:
data = Prep_Data(path=base_path+"vary_1.4_0.8_.pkl")
dvels = data.dvs
rknee = savgol_filter(data.states.T[4], 401, 3)
lknee = savgol_filter(data.states.T[6], 401, 3)

In [105]:
def make_vel_plot(vels, dvels):
    sns.set_style("darkgrid")
    sns.set_context("paper", font_scale=2)
    blues = sns.dark_palette("blue", 8)
    
    f, ax = plt.subplots(figsize=(10,5))
    times = np.arange(vels.shape[0])*0.002
    ax.plot(np.arange(dvels.shape[0])*0.002, dvels, "--", color=blues[3], linewidth=4, label="desired velocity")
    ax.plot(times, vels, "-", color=blues[7], linewidth=1, label="hip velocity")
    if vels.shape[0] > 2:
        ax.plot(times[vels.shape[0]-1], vels[-1], "o", color=blues[7], linewidth=6, label="current hip velocity")
    else:
        ax.plot(0., 0., "o", color=blues[7], linewidth=6, label="current hip velocity")
    ax.set_ylim(0., 1.5)
    ax.set_xlim(0, 21)
    ax.legend(loc=4)
    
    f.text(0.5, 0.05, 'time(s)', ha='center')
    f.text(0.05, 0.5, 'hip velocity (m/s)', va='center', rotation='vertical')
    
    return f

In [106]:
video_path = base_path + "frames/"
if not os.path.exists(video_path):
    os.makedirs(video_path)
for i in range(10000):
    if i % 25 == 0:
        fig = make_vel_plot(data.vels_smooth[0:i], dvels)
        plt.savefig(video_path+str("%05d" % i)+".jpg")
        plt.clf()
        img = imageio.imread(video_path+str("%05d" % i)+".jpg")
        img = cv2.resize(img, (500, 250))
        frame = np.concatenate([images[i], img], axis=0)
        imageio.imwrite(video_path+str("%05d" % i)+".jpg", frame)
    
#     vel_images += []
# vel_images = np.array(vel_images)
# print vel_images.shape[0]

In [107]:
video_path = base_path + "frames/"
if not os.path.exists(video_path):
    os.makedirs(video_path)
frames = []
for i in range(10000):
    if i % 25 == 0:
        frames += [imageio.imread(video_path+str("%05d" % i)+".jpg")]
frames = np.array(frames)
print frames.shape


(400, 750, 500, 3)


In [108]:
frames = list(frames)

In [109]:
clip = ImageSequenceClip(frames, fps=20)
clip.write_videofile(base_path+base_path.split("/")[3]+"demo.mp4", audio=False)

[MoviePy] >>>> Building video log/2000/data/vary_1.4_0.8_/vary_1.4_0.8_demo.mp4
[MoviePy] Writing video log/2000/data/vary_1.4_0.8_/vary_1.4_0.8_demo.mp4


100%|██████████| 401/401 [00:01<00:00, 314.39it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: log/2000/data/vary_1.4_0.8_/vary_1.4_0.8_demo.mp4 

